In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [56]:
import os
from datetime import date

from github import Github
from tqdm import tqdm
import pandas as pd

In [5]:
g = Github(os.environ['GITHUB_TOKEN'], timeout=45)

In [59]:
%%time
data = []
repos = g.search_repositories('stars:>=100000')
for repo in tqdm(repos, total=repos.totalCount):
    try:
        repo_info = {}
        repo_info['id'] = repo.id
        repo_info['url'] = repo.url
        repo_info['name'] = repo.name
        repo_info['owner'] = repo.owner.login
        repo_info['stars'] = repo.stargazers_count
        repo_info['commits'] = []
        contributors = repo.get_contributors()
        for contributor in tqdm(contributors, total=contributors.totalCount):
            repo_info['commits'].append({
                'author_id': contributor.id,
                'author_url': contributor.url,
                'author_location': contributor.location,
                'total_commits': repo.get_commits(author=contributor).totalCount
            })
            break
        data.append(repo_info)
        break
    except Exception as e:
        print(e)

  0%|                                                    | 0/38 [00:02<?, ?it/s]

CPU times: user 63.6 ms, sys: 7.65 ms, total: 71.3 ms
Wall time: 3.02 s


In [60]:
df = pd.json_normalize(data, record_path='commits', meta=['id', 'url', 'name', 'owner', 'stars'], record_prefix='commit_')

In [61]:
df.to_csv(f'../data/conributors_{date.today()}.csv', index=False)